# Lab: Clustering Handwritten Digits
Let's try out some of our clustering exercises on our digits dataset (the original MNIST dataset we used with KNN).

Let's first fetch the dataset from the internet (which may take a while, note the note):

In [ ]:
from __future__ import print_function

In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
%matplotlib inline

In [ ]:
from sklearn import metrics
from sklearn.metrics import pairwise_distances
from sklearn.metrics import silhouette_score
from sklearn.preprocessing import scale

In [ ]:
from sklearn.datasets import fetch_mldata
from sklearn.cluster import KMeans
from sklearn.utils import shuffle

In [ ]:
mnist = fetch_mldata("MNIST Original")
X_digits=mnist.data
Y_digits=mnist.target
# Note: if you here crash here on run-all, run incrementally as the data can take a while to load

In [ ]:
X_digits.shape # 28x28 image files

## Plot Data

In [ ]:
plt.rc("image", cmap="binary") # use black/white palette for plotting
minimum = 20000 # change this to see different slices of the full set
for i in range(minimum, minimum+10):
    plt.subplot(2,5,i+1-minimum)
    plt.imshow(X_digits[i].reshape(28,28))
    plt.xticks(())
    plt.yticks(())
    print (Y_digits[i])
plt.tight_layout()

In [ ]:
X_digits, Y_digits = shuffle(X_digits, Y_digits) # shuffle dataset (which is ordered!)
X_digits = X_digits[-5000:]       # take only the last instances, to shorten runtime of KMeans

Let's have a look at some of the instances in the dataset we just loaded:

In [ ]:
plt.rc("image", cmap="binary") # use black/white palette for plotting
for i in range(10):
    plt.subplot(2,5,i+1)
    plt.imshow(X_digits[i].reshape(28,28))
    plt.xticks(())
    plt.yticks(())
plt.tight_layout()

In [ ]:
# let's look at our raw data
X_digits[0].reshape(28,28)

In [ ]:
# that's messy, so let's use pandas and string trickery to visualize and format our input matrix
print(pd.DataFrame(X_digits[0].reshape(28,28)).to_string())

# looks like our jpg number!

In [ ]:
# let's look at our supervised classes
Y_digits[0:11]

>Notice we can read the number easily in the input matrix, but can clustering find similarities between 7's, and contrast between 7's and 8's?

## K=20

Try creating a KMeans clusterer with 20 classes (obviously 10 would be ideal, but let's try 20 first).  Fit the model to the digits data.

In [ ]:
km = KMeans(n_clusters=20)

In [ ]:
km.fit(X_digits)

Store the means of the clusters (the centroids) in a variable called `mu_digits` via a call to `cluster_centers_`:

In [ ]:
mu_digits = km.cluster_centers_

In [ ]:
mu_digits.shape
# we have 20 clusters, so 20 centroids, and each centroid has an average value of its related points in 784 (28*28) dimensions

In [ ]:
type(km.labels_)
#km.labels_
# are these all equally represented?

In [ ]:
km.labels_.min()

Now let's take a look at those "mean digits".  What is going on here?

In [ ]:
plt.figure(figsize=(16,6))
for i in range(2*(mu_digits.shape[0]//2)): # loop over all means
    plt.subplot(2,mu_digits.shape[0]/2,i+1)
    plt.imshow(mu_digits[i].reshape(28,28))
    plt.xticks(())
    plt.yticks(())
plt.tight_layout()

## K=10

Let's compare with the common-sense approach of ten clusters.

In [ ]:
km = KMeans(n_clusters=10)

In [ ]:
km.fit(X_digits)

In [ ]:
mu_digits = km.cluster_centers_

In [ ]:
mu_digits.shape

In [ ]:
plt.figure(figsize=(16,6))
for i in range(2*(mu_digits.shape[0]//2)): # loop over all means
    plt.subplot(2,mu_digits.shape[0]/2,i+1)
    plt.imshow(mu_digits[i].reshape(28,28))
    plt.xticks(())
    plt.yticks(())
plt.tight_layout()

Now that you've tried 20 and 10 clusters, let's see where the optimum is. Try building a loop that tries out different values of k and stores the resulting silhouette coefficients.  What do you find the optimum k value to be?  Is it 10 as we might hope?

In [ ]:
def get_cluster_centers(X, labels, k_num):
    CC_list = []
    for k in range(k_num):
        # get the mean coordinates of each cluster
        CC_list.append(np.mean(X[labels == k], axis = 0))
    return CC_list

# for each cluster substract the mean from each data point to get the error
# then get the magnitude of each error, square it, and sum it
def get_SSE(X, labels):
    k_num = len(np.unique(labels))
    CC_list = get_cluster_centers(X, labels, k_num)
    CSEs = []
    for k in range(k_num):
        # for each cluster of k we get the coordinates of how far off each point is to the cluster
        error_cords = X[labels == k] - CC_list[k]
        # square the coordinates and sum to get the magnitude squared
        error_cords_sq = error_cords ** 2
        error_mag_sq = np.sum(error_cords_sq, axis = 1)
        # since we already have the magnitude of the error squared we can just take the sum for the cluster
        CSE = np.sum(error_mag_sq)
        CSEs.append(CSE)
    # sum each cluster's sum of squared errors
    return sum(CSEs)

In [ ]:
SSEs = []
Sil_coefs = []
for k in range(2,20):
    km = KMeans(n_clusters=k, random_state=1)
    km.fit(X_digits)
    labels = km.labels_
    Sil_coefs.append(metrics.silhouette_score(X_digits, labels, metric='euclidean'))
    SSEs.append(get_SSE(X_digits, labels)) # The SSE is just inertia, we could have just said km.inertia_

In [ ]:
fig, (ax1, ax2) = plt.subplots(1,2, figsize=(15,5), sharex=True)
k_clusters = range(2,20)
ax1.plot(k_clusters, Sil_coefs)
ax1.set_xlabel('number of clusters')
ax1.set_ylabel('silhouette coefficient')
plt.xticks(np.arange(2, 20, step=2))

# plot Intertia/SSE on ax2
ax2.plot(k_clusters, SSEs)
ax2.set_xlabel('number of clusters')
ax2.set_ylabel('SSE');

Another [sklearn example](http://scikit-learn.org/stable/auto_examples/cluster/plot_kmeans_digits.html) of clustering MNIST, with PCA, more metrics, and visualizations.

>NOTE: we don't need to scale this example, because all features are in the same units. If you do scale, normalize to keep positive numbers. Standardized negatives don't work with grayscale plots.